In [3]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
import os

In [ ]:
# 🔹 Chargement des données avec augmentation
'''
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # 80% train / 20% validation
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)



datagen = ImageDataGenerator(
    rescale=1./255,  # Normalisation
    rotation_range=20,  # Rotation aléatoire de 20°
    width_shift_range=0.2,  # Décalage horizontal de 20%
    height_shift_range=0.2,  # Décalage vertical de 20%
    horizontal_flip=True  # Miroir horizontal
)

train_data = datagen.flow_from_directory(
    'dataset/train/',  # Dossier contenant 'chat/' et 'oiseau/'
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

val_data = datagen.flow_from_directory(
    'dataset/validation',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

'''

# 🔹 Définition des paramètres
IMG_SIZE = (224, 224)  # Taille standard pour ResNet
BATCH_SIZE = 32
EPOCHS = 10  # Augmenter si dataset plus grand


# 📌 Générateur pour l'entraînement (avec Data Augmentation)
datagen = ImageDataGenerator(
    rescale=1./255,  # Normalisation des pixels entre 0 et 1
    rotation_range=20,  # Rotation aléatoire de 20°
    width_shift_range=0.2,  # Décalage horizontal de 20%
    height_shift_range=0.2,  # Décalage vertical de 20%
    horizontal_flip=True  # Miroir horizontal
)

# 📌 Générateur pour la validation (PAS de Data Augmentation, juste normalisation)
val_datagen = ImageDataGenerator(rescale=1./255)

train_data = datagen.flow_from_directory(
    'dataset/train/',  # Dossier contenant 'chat/' et 'oiseau/'
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)


val_data = datagen.flow_from_directory(
    'dataset/validation',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)



In [ ]:
# 🔹 Chargement du modèle ResNet50 pré-entraîné
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 🔹 On empêche le backbone d’être modifié au début
base_model.trainable = False

# 🔹 Construction du modèle
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Réduit les dimensions
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)  # Réduit l'overfitting
output = Dense(1, activation='sigmoid')(x)  # Sigmoid pour classification binaire

model = Model(inputs=base_model.input, outputs=output)

# 🔹 Compilation du modèle
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:

# 🔹 Entraînement du modèle
model.fit(train_data, validation_data=val_data, epochs=EPOCHS)

In [ ]:
# 🔹 Sauvegarde du modèle pour Django
model.save('chat_chien_model.h5')